PREPROCESADO DE DATOS:

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np

In [2]:

# Cargar CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train.astype('float32') / 255.0  # Normalizar imágenes a [0,1]
x_test = x_test.astype('float32') / 255.0

# Dividir en datos etiquetados y no etiquetados (80% sin etiquetas)
np.random.seed(42)
unlabeled_indices = np.random.choice(len(x_train), size=int(0.8 * len(x_train)), replace=False)
labeled_indices = np.setdiff1d(np.arange(len(x_train)), unlabeled_indices)

x_train_labeled = x_train[labeled_indices]
y_train_labeled = y_train[labeled_indices]
x_train_unlabeled = x_train[unlabeled_indices]

print(f"Etiquetadas: {x_train_labeled.shape}, No etiquetadas: {x_train_unlabeled.shape}")

Etiquetadas: (10000, 32, 32, 3), No etiquetadas: (40000, 32, 32, 3)


CREACIÓN DE LA RED NEURONAL CONVOLUCIONAL

In [3]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [4]:

def crear_modelo():
    model = models.Sequential()

    # Bloque convolucional 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005), input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Bloque convolucional 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Bloque convolucional 3
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Aplanar
    model.add(layers.Flatten())

    # Bloque denso 1
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.6))

    # Capa de salida
    model.add(layers.Dense(100, activation='softmax'))

    # Compilación del modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:

# Crear modelo
model = crear_modelo()

# Callbacks para mejorar el entrenamiento
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
]

# Entrenamiento del modelo
history = model.fit(
    x_train_labeled, y_train_labeled,
    epochs=20,
    batch_size=256,
    validation_data=(x_test, y_test),
    callbacks=callbacks
)

In [ ]:
# Evaluamos el rendimiento en el conjunto de prueba
train_loss, train_acc = model.evaluate(x_train_labeled, y_train_labeled, verbose=0)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Self-training: Train Accuracy: {:.4f} - Test Accuracy: {:.4f}".format(train_acc, test_acc))